In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

In [ ]:
column_names = [
    'id', 'label', 'statement', 'subject', 'speaker', 'speaker_job_title', 
    'state', 'party', 'barely_true_counts', 'false_counts', 'half_true_counts', 
    'mostly_true_counts', 'pants_on_fire_counts', 'context'
]

# Đọc file TSV với tên cột đã xác định
data = pd.read_csv('/kaggle/input/dataset-original/Dataset_Original/LIAR/train.tsv', sep='\t', names=column_names, header=None)

# Data overview

In [ ]:
# Data exploration
data.head()

In [ ]:
print(data['label'].value_counts())

# Missing & duplicate values

In [ ]:
data.isnull().sum()

In [ ]:
num_unique_values = data['subject'].nunique()
print(f'{num_unique_values}/{data.shape[0]}')

In [ ]:
# Number of unique values in text
num_unique_values = data['statement'].nunique()
print(f'{num_unique_values}/{data.shape[0]}')

In [ ]:
# Kiểm tra các giá trị không phải là chuỗi trong cột 'subject'
non_string_subject = data['subject'].apply(lambda x: not isinstance(x, str))
print(data[non_string_subject])
# Kiểm tra và đếm số giá trị không phải chuỗi trong cột 'subject'
non_string_subject_count = non_string_subject.sum()
print("Count of values that not a string value in 'subject':", non_string_subject_count)


# Distribution of speaker length

In [ ]:
# Kiểm tra các giá trị không phải là chuỗi trong cột 'speaker'
non_string_speaker = data['speaker'].apply(lambda x: not isinstance(x, str))
print(data[non_string_speaker])
# Kiểm tra và đếm số giá trị không phải chuỗi trong cột 'speaker'
non_string_speaker_count = non_string_speaker.sum()
print("Count of values that not a string value in 'speaker':", non_string_speaker_count)


In [ ]:
is_string_speakers = data['speaker'].apply(lambda x: isinstance(x, str))
data['speaker_length'] = data['speaker'][is_string_speakers].apply(len)
print(data['speaker_length'].describe())

In [ ]:
print(data[data['speaker_length']==3]['speaker'])

In [ ]:
print(data[data['speaker_length']==50]['speaker'])

# Distribution of statement length

In [ ]:
is_string_statements = data['statement'].apply(lambda x: isinstance(x, str))
# Tính độ dài của mỗi statement và thêm vào DataFrame dưới dạng cột mới 'statement_length'
data['statement_length'] = data['statement'][is_string_statements].apply(len)

# In ra các thống kê về độ dài của các tiêu đề
print(data['statement_length'].describe())

In [ ]:
print(data[data['statement_length']==11]['statement'])

In [ ]:
print(data[data['statement_length']==3204]['statement'])

In [ ]:
# Define a pattern for JavaScript code
pattern = r'<script.*?</script>'

# Find examples that contain JavaScript code
js_examples = data[data['statement'].str.contains(pattern, regex=True, na=False)]

# Print the first few examples
print(js_examples.head())

In [ ]:
# Describe 'sentence_length' for label pants-fire
print(data[data['label'] == 'pants-fire']['statement_length'].describe())

# Describe 'sentence_length' for label false
print(data[data['label'] == 'false']['statement_length'].describe())

# Describe 'sentence_length' for label half-true
print(data[data['label'] == 'half-true']['statement_length'].describe())

# Describe 'sentence_length' for label mostly-true
print(data[data['label'] == 'mostly-true']['statement_length'].describe())

# Describe 'sentence_length' for label true
print(data[data['label'] == 'true']['statement_length'].describe())


## Below 20 characters

In [ ]:
plt.hist(data['statement_length'], bins=50, range=[11, 20])
plt.title('Distribution of text length below 20 characters')
plt.xlabel('Text length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print(data[data['statement_length']<20])

## From 20 to 100

In [ ]:
plt.hist(data['statement_length'], bins=50, range=[20, 100])
plt.title('Distribution of text length from 20 to 100 characters')
plt.xlabel('Text length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print(data[data['statement_length']==76]['statement'])

## Above 100 characters

In [ ]:
plt.hist(data['statement_length'], bins=50, range=[100, 1000])
plt.title('Distribution of text length from 100 to 1000 characters')
plt.xlabel('Text length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print(data[data['statement_length']==100]['statement'])

## Above 1000 characters

In [ ]:
plt.hist(data['statement_length'], bins=50, range=[1000, 10000])
plt.title('Distribution of text length from 1000 to 10000 characters')
plt.xlabel('Text length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Filter the DataFrame for examples where the text below 100 characters
long_texts = data[data['statement'].str.len() < 100]

# Get the label distribution
label_distribution = long_texts['label'].value_counts()

print(label_distribution)

In [ ]:
# Filter the DataFrame for examples where the text exceeds 10000 characters
long_texts = data[(data['statement'].str.len() >= 100) & (data['statement'].str.len() <= 1000)]
# Get the label distribution
label_distribution = long_texts['label'].value_counts()

print(label_distribution)

In [ ]:
# Filter the DataFrame for examples where the text below 100 characters
long_texts = data[data['statement'].str.len() > 1000]

# Get the label distribution
label_distribution = long_texts['label'].value_counts()

print(label_distribution)

## Correlation of Speaker's statement credit and 

In [ ]:
# Tạo biến tổng số phát biểu
data['total_statements'] = data.iloc[:, 8:13].sum(axis=1)

# Tạo các biến tỷ lệ
data['barely_true_ratio'] = data['barely_true_counts'] / data['total_statements']
data['false_ratio'] = data['false_counts'] / data['total_statements']
data['half_true_ratio'] = data['half_true_counts'] / data['total_statements']
data['mostly_true_ratio'] = data['mostly_true_counts'] / data['total_statements']
data['pants_on_fire_ratio'] = data['pants_on_fire_counts'] / data['total_statements']
# Kiểm tra lại vài dòng đầu tiên sau khi thêm các biến mới
data.head()


In [ ]:
unique_labels = data['label'].unique()
print(unique_labels)

In [ ]:
# Tính tần suất của từng nhãn
label_counts = data['label'].value_counts()
print(label_counts)

In [ ]:
# Cập nhật từ điển chuyển đổi để bao gồm tất cả các nhãn
label_to_numeric = {
    'pants-fire': 0,
    'false': 1,
    'half-true': 2,
    'mostly-true': 3,
    'true': 4,
    'barely-true': 5  # Giả sử 'barely-true' là nhãn mới mà bạn phát hiện ra
}

# Chuyển đổi nhãn thành số
data['label_numeric'] = data['label'].apply(lambda x: label_to_numeric.get(x, -1))  # -1 cho các giá trị không xác định

# Kiểm tra lại để đảm bảo chuyển đổi thành công
print(data[['label', 'label_numeric']].head())


In [ ]:
# Tính tương quan
correlation_matrix = data[['barely_true_ratio', 'false_ratio', 'half_true_ratio', 'mostly_true_ratio', 'pants_on_fire_ratio', 'label_numeric']].corr()
print(correlation_matrix)

# Biểu đồ nhiệt (heatmap) của ma trận tương quan
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Credit History Ratios and Label')
plt.show()


In [ ]:
def calculate_trustworthiness(row):
    total_statements = (
        row['barely_true_counts'] + row['false_counts'] + 
        row['half_true_counts'] + row['mostly_true_counts'] + 
        row['pants_on_fire_counts']
    )
    if total_statements == 0:
        return 0
    trustworthiness = (row['true_counts'] + row['mostly_true_counts']) / total_statements
    return trustworthiness

# Đảm bảo cột 'true_counts' được tính từ nhãn 'true'
data['true_counts'] = data['label'].apply(lambda x: 1 if x == 'true' else 0)

# Tính tổng các phát biểu theo người phát ngôn
speaker_group = data.groupby('speaker').sum().reset_index()

# Tính độ uy tín
speaker_group['trustworthiness'] = speaker_group.apply(calculate_trustworthiness, axis=1)

# Hiển thị độ uy tín
print(speaker_group[['speaker', 'trustworthiness']])